## As we want to use sklearn for a end to end process, we will be creating our custom transformer for preprocessing

In [8]:
#sklearn libraries
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

import pickle

In [9]:
raw_df = pd.read_csv("../data/pr_trans.csv")
raw_df.head(3)

,S/N,Project Name,Street Name,Type,Postal District,Market Segment,Tenure,Type of Sale,No. of Units,Price ($),Nett Price ($),Area (Sqft),Type of Area,Floor Level,Unit Price ($psf),Date of Sale
0,1,24 ONE RESIDENCES,PASIR PANJANG ROAD,Apartment,5,RCR,Freehold,New Sale,1.0,1383800.0,-,807.0,Strata,01 to 05,1714.0,Sep-17
1,2,24 ONE RESIDENCES,PASIR PANJANG ROAD,Apartment,5,RCR,Freehold,Resale,1.0,818000.0,-,484.0,Strata,01 to 05,1689.0,Aug-21
2,3,24 ONE RESIDENCES,PASIR PANJANG ROAD,Apartment,5,RCR,Freehold,New Sale,1.0,870100.0,-,484.0,Strata,01 to 05,1796.0,Aug-17


In [10]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117344 entries, 0 to 117343
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   S/N                117344 non-null  int64  
 1   Project Name       117344 non-null  object 
 2   Street Name        117344 non-null  object 
 3   Type               117344 non-null  object 
 4   Postal District    117344 non-null  int64  
 5   Market Segment     117344 non-null  object 
 6   Tenure             117344 non-null  object 
 7   Type of Sale       117344 non-null  object 
 8   No. of Units       117344 non-null  float64
 9   Price ($)          117344 non-null  float64
 10  Nett Price ($)     117344 non-null  object 
 11  Area (Sqft)        117344 non-null  float64
 12  Type of Area       117344 non-null  object 
 13  Floor Level        117344 non-null  object 
 14  Unit Price ($psf)  117344 non-null  float64
 15  Date of Sale       117344 non-null  object 
dtypes:

## Changing sale_date to sale_year, sale_month

In [11]:
class SaleDate2Yearmonth(TransformerMixin, BaseEstimator):
    def __init__(self, sale_date_col='Date of Sale'):
        self.sale_date_col = sale_date_col
        
    def fit(self, df, y=None):
        return self
    
    def transform(self, df, y=None):
        df = df.copy()
        sale_date = pd.to_datetime(df[self.sale_date_col], format='%b-%y')
        df['sale_month'] = sale_date.dt.month
        df['sale_year'] = sale_date.dt.year
        df = df.drop(self.sale_date_col, axis=1)
        return df

## remaining lease to remaining lease months

In [15]:
class RemainingLease(TransformerMixin, BaseEstimator):
    from datetime import datetime
    def __init__(self, Tenure_col='Tenure'):
        self.Tenure_col = Tenure_col
        
    def fit(self,X, y=None):
        return self
    
    def __years_to_yrs(self,year_list):
        try:
            year_list = [item for item in year_list if item] # remove none
        except Exception as e:
            print(year_list)
            return np.nan

 
        if len(year_list)==1:
            total_yrs = 999999
            return total_yrs
          
        try:
            total_yrs = int(year_list[0]) - int(datetime.now().year)+int(year_list[-1])
        except:
            total_yrs = int(year_list[0])
        return total_yrs
    
    def transform(self, df, y=None):
        lease_yrs = df[self.Tenure_col].astype(str).str.split(" ")
        df['tenure_yrs'] = lease_yrs.map(self.__years_to_yrs).astype(int)
        df = df.drop('Tenure', axis=1)
        return df

## Columns to drop

In [18]:
class DropColumns(TransformerMixin, BaseEstimator):
    def __init__(self, drop_columns=['S/N', 'Project Name', 'Street Name', 'Nett Price ($)']):
        self.drop_columns = drop_columns
    
    def fit(self,X,y=None):
        return self
    
    def transform(self,df, y=None):
        return df.drop(self.drop_columns, axis=1)

## Change Column Names

In [19]:
class ChangeColNames(TransformerMixin, BaseEstimator):
    def __init__(self, col_names=['type', 'post_district', 'market_segment', 'type_sale', 
                                  'no_units', 'price', 'area', 'type_area', 'floor_level', 'unit_price',
                                  'sale_month', 'sale_year', 'tenure_yrs']):
        self.col_names = col_names
    
    def fit(self,X,y=None):
        return self
    
    def transform(self,df, y=None):
        df.columns=self.col_names
        return df

## Full_preprocessing_pipeline

In [20]:
preprocessing_pipeline = Pipeline(steps=[
    ('cvt_sale_date',SaleDate2Yearmonth(sale_date_col="Date of Sale")),
    ("cvt_remaining_lease", RemainingLease(Tenure_col='Tenure')),
    ('drop_cols', DropColumns()),
    ('change_col_names', ChangeColNames())
])
processed_df = preprocessing_pipeline.fit_transform(raw_df)
processed_df

,type,post_district,market_segment,type_sale,no_units,price,area,type_area,floor_level,unit_price,sale_month,sale_year,tenure_yrs
0,Apartment,5,RCR,New Sale,1.0,1383800.0,807.0,Strata,01 to 05,1714.0,9,2017,999999
1,Apartment,5,RCR,Resale,1.0,818000.0,484.0,Strata,01 to 05,1689.0,8,2021,999999
2,Apartment,5,RCR,New Sale,1.0,870100.0,484.0,Strata,01 to 05,1796.0,8,2017,999999
3,Apartment,5,RCR,New Sale,1.0,877200.0,484.0,Strata,01 to 05,1811.0,8,2017,999999
4,Apartment,5,RCR,New Sale,1.0,870100.0,484.0,Strata,01 to 05,1796.0,8,2017,999999
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117339,Executive Condominium,27,OCR,New Sale,1.0,693800.0,947.0,Strata,01 to 05,732.0,12,2016,99
117340,Executive Condominium,27,OCR,New Sale,1.0,734400.0,883.0,Strata,06 to 10,832.0,12,2016,99
117341,Executive Condominium,27,OCR,New Sale,1.0,623500.0,775.0,Strata,01 to 05,805.0,12,2016,99
117342,Executive Condominium,27,OCR,New Sale,1.0,621500.0,775.0,Strata,01 to 05,802.0,12,2016,99


## Exporting data for training

In [23]:
processed_df.to_csv("../data/pr_processed_df.csv", index=False, encoding='utf-8')

In [24]:
test_df = processed_df.copy()
test_df['sale_month']=6
test_df['sale_year']=2022
test_df.to_csv("../data/pr_test_df.csv", index=False, encoding='utf-8')